# Emergence of Latent Binary Encoding in Deep Neural Network Classifiers
---

### Supplementary material to reproduce results

All plots shown in the manuscript can be made using this notebook. Results are loaded from two 'best_results.pkl' files which contain data relative to the CIFAR10 and CIFAR100 dataset, correspondingly. 
Each experiment consists of running multiple trainings for different learning rates, and the training which has the best accuracy on the training set at the last epoch is picked. Experiments can be run on a GPU cluster using a Slurm script that we provide in 'emergence_binary_encoding/scripts.sh'. Best results for each experiment can then be selected using the Python script that we provide in 'emergence_binary_encoding/find_best_results.py'

In [ ]:
import torchvision
import torch
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import multivariate_normal
import pickle
from sklearn.mixture import GaussianMixture
from scipy.stats import norm
import scipy
import torch
import torch.nn as nn
from scipy.spatial import distance
import seaborn as sns
import yaml
import glob
import os
import matplotlib
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture


In [ ]:
dataset_dir = '../results'
with open(dataset_dir + '/cifar10/best_results.pkl', 'rb') as file:
    results_10 = pickle.load(file)

dataset_dir = '../results'
with open(dataset_dir + '/cifar100/best_results.pkl', 'rb') as file:
    results_100 = pickle.load(file)

In [ ]:
epochs = results_100['bin_enc']['training_hypers']['epochs'][0]
logging = results_100['bin_enc']['training_hypers']['logging'][0]

In [ ]:
sns.set(style="whitegrid")
alpha=0.3

In [ ]:
color_bin = '#1f77b4'
color_lin = '#2ca02c'
color_no = '#ff7f0e'
color_nonlin = '#d62728'

### Figure 1

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(10, 8), sharex=True)

color_0 = color_bin
color_1 = color_lin

label_size = 18
legend_size = 12

x = np.arange(logging, epochs+logging,logging)

y = np.mean(results_10['bin_enc']['binarity_train']['score'], axis=1)
std = np.std(results_10['bin_enc']['binarity_train']['score'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[0][0], color=color_0, label='BinEnc')
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_0)

y = np.mean(results_10['lin_pen']['binarity_train']['score'], axis=1)
std = np.std(results_10['lin_pen']['binarity_train']['score'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[0][0], color=color_1, label='LinPen')
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_1)

y = np.mean(results_10['bin_enc']['binarity_train']['stds'], axis=1)
std = np.std(results_10['bin_enc']['binarity_train']['stds'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[1][0], color=color_0)
axes[1][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_0)

y = np.mean(results_10['lin_pen']['binarity_train']['stds'], axis=1)
std = np.std(results_10['lin_pen']['binarity_train']['stds'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[1][0], color=color_1)
axes[1][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_1)


y = np.mean(results_10['bin_enc']['binarity_train']['peaks_distance_mean'], axis=1)
std = np.std(results_10['bin_enc']['binarity_train']['peaks_distance_mean'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[2][0], color=color_0)
axes[2][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_0)

y = np.mean(results_10['lin_pen']['binarity_train']['peaks_distance_mean'], axis=1)
std = np.std(results_10['lin_pen']['binarity_train']['peaks_distance_mean'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[2][0], color=color_1)
axes[2][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_1)


x = np.arange(logging, epochs+logging,logging)

y = np.mean(results_100['bin_enc']['binarity_train']['score'], axis=1)
std = np.std(results_100['bin_enc']['binarity_train']['score'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[0][1], color=color_0, label='BinEnc')
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_0)

y = np.mean(results_100['lin_pen']['binarity_train']['score'], axis=1)
std = np.std(results_100['lin_pen']['binarity_train']['score'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[0][1], color=color_1, label='LinPen')
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_1)


y = np.mean(results_100['bin_enc']['binarity_train']['stds'], axis=1)
std = np.std(results_100['bin_enc']['binarity_train']['stds'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[1][1], color=color_0)
axes[1][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_0)

y = np.mean(results_100['lin_pen']['binarity_train']['stds'], axis=1)
std = np.std(results_100['lin_pen']['binarity_train']['stds'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[1][1], color=color_1)
axes[1][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_1)


y = np.mean(results_100['bin_enc']['binarity_train']['peaks_distance_mean'], axis=1)
std = np.std(results_100['bin_enc']['binarity_train']['peaks_distance_mean'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[2][1], color=color_0)
axes[2][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_0)

y = np.mean(results_100['lin_pen']['binarity_train']['peaks_distance_mean'], axis=1)
std = np.std(results_100['lin_pen']['binarity_train']['peaks_distance_mean'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[2][1], color=color_1)
axes[2][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_1)


axes[0][0].set_title('CIFAR 10', size=label_size)
axes[0][1].set_title('CIFAR 100', size=label_size)

axes[2][0].set_xlabel('Epochs', size=label_size)
axes[2][1].set_xlabel('Epochs', size=label_size)
axes[0][0].set_ylabel('Score', size=label_size)
axes[1][0].set_ylabel('Std', size=label_size)
axes[2][0].set_ylabel('Peaks distance', size=label_size)

axes[2][0].set_xticks(range(20, 201, 20))

axes[0][0].legend(loc='upper left', fontsize=legend_size)
axes[0][1].legend(loc='upper left', fontsize=legend_size)

axes[0][1].set_ylim(-1.5,0)
axes[0][0].sharey(axes[0][1])

axes[1][1].set_ylim(0,1)
axes[1][0].sharey(axes[1][1])


axes[2][1].set_ylim(0,18)
axes[2][0].sharey(axes[2][1])

plt.tight_layout()
plt.savefig('./score.png')

### Figure 2

In [ ]:
pen_10 = results_10['bin_enc']['penultimate_train'][:,0]
gmm = GaussianMixture(n_components=2)
scaler = StandardScaler()
scaler.fit(pen_10.reshape(-1,1))
pen_layer_scaled_10 = scaler.transform(pen_10.reshape(-1,1))
gmm.fit(pen_layer_scaled_10)
score= gmm.score(pen_layer_scaled_10)
means = gmm.means_.flatten()
std = np.mean(np.sqrt(gmm.covariances_).flatten())
peaks = np.abs(means[0]-means[1])/np.mean(std)

print('CIFAR 10')
print('Score: ', score)
print('Std: ', std)
print('Peaks distance: ', peaks)

In [ ]:
pen_100 = results_100['bin_enc']['penultimate_train'][:,0]
gmm = GaussianMixture(n_components=2)
scaler = StandardScaler()
scaler.fit(pen_100.reshape(-1,1))
pen_layer_scaled_100 = scaler.transform(pen_100.reshape(-1,1))
gmm.fit(pen_layer_scaled_100)
score= gmm.score(pen_layer_scaled_100)
means = gmm.means_.flatten()
std = np.mean(np.sqrt(gmm.covariances_).flatten())
peaks = np.abs(means[0]-means[1])/np.mean(std)

print('CIFAR 100')
print('Score: ', score)
print('Std: ', std)
print('Peaks distance: ', peaks)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

label_size = 18
legend_size = 12

axes[0].hist(pen_layer_scaled_10, bins=100);
axes[1].hist(pen_layer_scaled_100, bins=100);

axes[0].set_title('CIFAR 10', size=label_size)
axes[1].set_title('CIFAR 100', size=label_size)
axes[0].set_xlabel('z', size=label_size)
axes[1].set_xlabel('z', size=label_size)

plt.tight_layout()
plt.savefig('bins.png')

### Figure 3

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5.1),  sharey=True)

label_size = 18
legend_size = 12

x = np.arange(logging, epochs+logging,logging)

y = np.mean(results_10['bin_enc']['same_encoding_fraction_train'], axis=1)
std = np.std(results_10['bin_enc']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[0], color=color_bin)
axes[0].fill_between(x, y - std, y + std, alpha=alpha, color=color_bin)

y = np.mean(results_10['lin_pen']['same_encoding_fraction_train'], axis=1)
std = np.std(results_10['lin_pen']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', color=color_lin, ax=axes[0])
axes[0].fill_between(x, y - std, y + std, alpha=alpha, color=color_lin)

y = np.mean(results_10['nonlin_pen']['same_encoding_fraction_train'], axis=1)
std = np.std(results_10['nonlin_pen']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[0], color=color_nonlin)
axes[0].fill_between(x, y - std, y + std, alpha=alpha, color=color_nonlin)

y = np.mean(results_10['no_pen']['same_encoding_fraction_train'], axis=1)
std = np.std(results_10['no_pen']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[0], color=color_no)
axes[0].fill_between(x, y - std, y + std, alpha=alpha, color=color_no)


x = np.arange(logging, epochs+logging,logging)

y = np.mean(results_100['bin_enc']['same_encoding_fraction_train'], axis=1)
std = np.std(results_100['bin_enc']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[1], color=color_bin)
axes[1].fill_between(x, y - std, y + std, alpha=alpha, color=color_bin)

y = np.mean(results_100['lin_pen']['same_encoding_fraction_train'], axis=1)
std = np.std(results_100['lin_pen']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', color=color_lin, ax=axes[1])
axes[1].fill_between(x, y - std, y + std, alpha=alpha, color=color_lin)

y = np.mean(results_100['nonlin_pen']['same_encoding_fraction_train'], axis=1)
std = np.std(results_100['nonlin_pen']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[1], color=color_nonlin)
axes[1].fill_between(x, y - std, y + std, alpha=alpha, color=color_nonlin)

y = np.mean(results_100['no_pen']['same_encoding_fraction_train'], axis=1)
std = np.std(results_100['no_pen']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[1], color=color_no)
axes[1].fill_between(x, y - std, y + std, alpha=alpha, color=color_no)

axes[0].set_title('CIFAR 10', size=label_size)
axes[1].set_title('CIFAR 100', size=label_size)

axes[0].set_xlabel('Epochs', size=label_size)
axes[1].set_xlabel('Epochs', size=label_size)
axes[0].set_ylabel('Fraction', size=label_size)

axes[0].set_xticks(range(20, 201, 20))

axes[1].get_legend().remove()

plt.tight_layout()
plt.savefig('./fraction.png')

### Figure 4

In [ ]:
x = np.arange(logging, epochs+logging,logging)

fig, axes = plt.subplots(3, 2, figsize=(10, 8), sharex=True)

y = np.mean(results_10['lin_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_10['lin_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', ax=axes[0][0],color=color_lin)
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_lin)

y = np.mean(results_100['lin_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_100['lin_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', ax=axes[0][1],color=color_lin)
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_lin)

y = np.mean(results_10['nonlin_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_10['nonlin_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[0][0], color=color_nonlin)
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_nonlin)

y = np.mean(results_10['no_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_10['no_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[1][0], color=color_no)
axes[1][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_no)

y = np.mean(results_100['no_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_100['no_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[1][1], color=color_no)
axes[1][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_no)

y = np.mean(results_100['nonlin_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_100['nonlin_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[0][1], color=color_nonlin)
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_nonlin)

y = np.mean(results_10['bin_enc']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_10['bin_enc']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='Binary ', ax=axes[2][0], color=color_bin)
axes[2][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_bin)

y = np.mean(results_100['bin_enc']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_100['bin_enc']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='Binary ', ax=axes[2][1], color=color_bin)
axes[2][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_bin)

axes[0][0].set_title('CIFAR 10', size=label_size)
axes[0][1].set_title('CIFAR 100', size=label_size)

axes[2][0].set_yscale('log')
axes[2][1].set_yscale('log')

axes[1][0].set_ylabel('Within-class covariance', size=label_size)

axes[2][0].set_xlabel('Epochs', size=label_size)
axes[2][1].set_xlabel('Epochs', size=label_size)

axes[2][0].set_xticks(range(20, 201, 20))
axes[2][1].set_xticks(range(20, 201, 20))


plt.savefig('./collapse.png')

plt.tight_layout()


### Figure 5

In [ ]:
x = np.arange(logging, epochs+logging,logging)

fig, axes = plt.subplots(2, 2, figsize=(10, 8), sharex=True)

y = np.mean(results_10['bin_enc']['accuracy_train'], axis=1)
std = np.std(results_10['bin_enc']['accuracy_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[0][0], color=color_bin)
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_bin)
y = np.mean(results_10['lin_pen']['accuracy_train'], axis=1)
std = np.std(results_10['lin_pen']['accuracy_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', color=color_lin, ax=axes[0][0])
axes[0][0].fill_between(x, y - std, y + std,color=color_lin,  alpha=alpha)
y = np.mean(results_10['nonlin_pen']['accuracy_train'], axis=1)
std = np.std(results_10['nonlin_pen']['accuracy_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[0][0], color=color_nonlin)
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_nonlin)
y = np.mean(results_10['no_pen']['accuracy_train'], axis=1)
std = np.std(results_10['no_pen']['accuracy_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[0][0], color=color_no)
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_no)

y = np.mean(results_10['bin_enc']['accuracy_test'], axis=1)
std = np.std(results_10['bin_enc']['accuracy_test'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[1][0], color=color_bin)
axes[1][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_bin)
y = np.mean(results_10['lin_pen']['accuracy_test'], axis=1)
std = np.std(results_10['lin_pen']['accuracy_test'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen',color=color_lin,  ax=axes[1][0])
axes[1][0].fill_between(x, y - std, y + std, color=color_lin, alpha=alpha)
y = np.mean(results_10['nonlin_pen']['accuracy_test'], axis=1)
std = np.std(results_10['nonlin_pen']['accuracy_test'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[1][0], color=color_nonlin)
axes[1][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_nonlin)
y = np.mean(results_10['no_pen']['accuracy_test'], axis=1)
std = np.std(results_10['no_pen']['accuracy_test'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[1][0], color=color_no)
axes[1][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_no)

y = np.mean(results_100['bin_enc']['accuracy_train'], axis=1)
std = np.std(results_100['bin_enc']['accuracy_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[0][1], color=color_bin)
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_bin)
y = np.mean(results_100['lin_pen']['accuracy_train'], axis=1)
std = np.std(results_100['lin_pen']['accuracy_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen',color=color_lin, ax=axes[0][1])
axes[0][1].fill_between(x, y - std, y + std,color=color_lin, alpha=alpha)
y = np.mean(results_100['nonlin_pen']['accuracy_train'], axis=1)
std = np.std(results_100['nonlin_pen']['accuracy_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[0][1], color=color_nonlin)
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_nonlin)
y = np.mean(results_100['no_pen']['accuracy_train'], axis=1)
std = np.std(results_100['no_pen']['accuracy_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[0][1], color=color_no)
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_no)

y = np.mean(results_100['bin_enc']['accuracy_test'], axis=1)
std = np.std(results_100['bin_enc']['accuracy_test'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[1][1], color=color_bin)
axes[1][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_bin)
y = np.mean(results_100['lin_pen']['accuracy_test'], axis=1)
std = np.std(results_100['lin_pen']['accuracy_test'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', color=color_lin, ax=axes[1][1])
axes[1][1].fill_between(x, y - std, y + std, color=color_lin, alpha=alpha)
y = np.mean(results_100['nonlin_pen']['accuracy_test'], axis=1)
std = np.std(results_100['nonlin_pen']['accuracy_test'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[1][1], color=color_nonlin)
axes[1][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_nonlin)
y = np.mean(results_100['no_pen']['accuracy_test'], axis=1)
std = np.std(results_100['no_pen']['accuracy_test'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[1][1], color=color_no)
axes[1][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_no)

axes[0][0].set_title('CIFAR 10', size=label_size)
axes[0][1].set_title('CIFAR 100', size=label_size)

axes[1][0].set_xlabel('Epochs', size=label_size)
axes[1][1].set_xlabel('Epochs', size=label_size)
axes[0][0].set_ylabel('Accuracy train set', size=label_size)
axes[1][0].set_ylabel('Accuracy test set', size=label_size)

axes[1,0].get_legend().remove()
axes[1,1].get_legend().remove()
axes[0][0].set_xticks(range(20, 201, 20))
axes[0][1].set_xticks(range(20, 201, 20))

plt.savefig('./accuracy.png')

plt.tight_layout()

### Table 1

In [ ]:
models = [ 'bin_enc',  'no_pen', 'lin_pen', 'nonlin_pen']
print('CIFAR10')
for model in models:
    print(model)
    print('test: ', np.mean(results_10[model]['accuracy_test'], axis=1)[-1], np.std(results_10[model]['accuracy_test'], axis=1)[-1])
    print('train: ', np.mean(results_10[model]['accuracy_train'], axis=1)[-1], np.std(results_10[model]['accuracy_train'], axis=1)[-1])

print('')
print('CIFAR100')      
for model in models:
    print(model)
    print('test: ', np.mean(results_100[model]['accuracy_test'], axis=1)[-1], np.std(results_100[model]['accuracy_test'], axis=1)[-1])
    print('train: ', np.mean(results_100[model]['accuracy_train'], axis=1)[-1], np.std(results_100[model]['accuracy_train'], axis=1)[-1])

### Table 2

In [ ]:
models = [ 'bin_enc',  'no_pen', 'lin_pen', 'nonlin_pen']

print('CIFAR10')
for model in models:
    print(model, ': ', np.around(np.mean(results_10[model]['mahalanobis_score']['auroc']),4), np.around(np.std(results_10[model]['mahalanobis_score']['auroc']),4))

print('')
print('CIFAR100')      
for model in models:
    print(model, ': ', np.around(np.mean(results_100[model]['mahalanobis_score']['auroc']),4), np.around(np.std(results_100[model]['mahalanobis_score']['auroc']),4))

### Table 3

In [ ]:
models = [ 'bin_enc',  'no_pen', 'lin_pen', 'nonlin_pen']

print('CIFAR10')
for model in models:
    print(model, ': ', np.around(np.mean(results_10[model]['perturbation_score']),4), np.around(np.std(results_10[model]['perturbation_score']),4))

print('')
print('CIFAR100')      
for model in models:
    print(model, ': ', np.around(np.mean(results_100[model]['perturbation_score']),4), np.around(np.std(results_100[model]['perturbation_score']),4))